In [1]:
import re
import typing as t
from collections import defaultdict
from pathlib import Path

import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.utils.data import Dataset, DataLoader, Subset, random_split

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
DATA_DIR = Path("data/")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE.upper()} device")

Using CUDA device


In [4]:
def on_cuda(device: str) -> bool:
    return device == "cuda"


def common_train(
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        train_dataloader: DataLoader,
        epochs: int,
        test_dataloader: DataLoader = None,
        verbose: int = 100,
        on_epoch_end: t.Callable[[], None] = None,
        device: str = "cpu",
) -> t.List[float]:
    train_losses = []
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n" + "-" * 32)
        train_loss = train_loop(
            train_dataloader,
            model,
            loss_fn,
            optimizer,
            verbose=verbose,
            device=device,
        )
        train_losses.append(train_loss.item())

        if test_dataloader:
            test_loop(test_dataloader, model, loss_fn, device=device)

        if on_epoch_end:
            on_epoch_end()

        print()
        torch.cuda.empty_cache()
    return train_losses


def train_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        verbose: int = 100,
        device: str = "cpu",
) -> torch.Tensor:
    model.train()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss = 0

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss
        if batch % verbose == 0:
            print(f"loss: {loss:>7f}  [{batch * len(x):>5d}/{size:>5d}]")

        del x, y, pred, loss
        torch.cuda.empty_cache()

    return avg_loss / num_batches


@torch.no_grad()
def test_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    avg_loss, num_batches = 0, len(dataloader)
    correct, total = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        avg_loss += loss_fn(pred, y)

        y_test = torch.flatten(y)
        y_pred = torch.flatten(pred.argmax(1))
        total += y_test.size(0)
        correct += (y_pred == y_test).sum()  # noqa

        del x, y, pred
        torch.cuda.empty_cache()

    avg_loss /= num_batches
    accuracy = correct / total
    print(f"Test Error: \n"
          f"\tAccuracy: {accuracy:>4f}, Loss: {avg_loss:>8f}")

    return avg_loss, accuracy


def train_test_split(dataset: t.Union[Dataset, t.Sized], train_part: float) -> t.Tuple[Subset, Subset]:
    train_size = round(train_part * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, lengths=(train_size, test_size))
    return train_dataset, test_dataset


@torch.no_grad()
def get_y_test_y_pred(
        model: nn.Module,
        test_dataloader: DataLoader,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    y_test = []
    y_pred = []
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        y_test.append(y)
        y_pred.append(pred)

        del x
        torch.cuda.empty_cache()

    return torch.flatten(torch.vstack(y_test).detach().cpu()), torch.flatten(torch.vstack(y_pred).detach().cpu())

## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.

  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```


Будем предсказывать каждую следующую букву в имени:

In [5]:
class NamesVocab:
    PAD = "<PAD>"
    PAD_IDX = 0
    UNK = "<UNK>"
    UNK_IDX = 1
    SOS = "<SOS>"
    SOS_IDX = 2
    EOS = "<EOS>"
    EOS_IDX = 3

    def __init__(self, names: t.List[str]):
        uniques = set()
        max_len = 0
        for name in map(str.lower, names):
            uniques.update(name)
            max_len = max(len(name), max_len)

        self.alphabet = [self.PAD, self.UNK, self.SOS, self.EOS, *uniques]
        self.max_len = max_len + 2  # место для <SOS> и <EOS>

        ch2i = {ch: i for i, ch in enumerate(self.alphabet)}
        self.ch2i = defaultdict(lambda: self.UNK_IDX, ch2i)

    def __len__(self):
        return len(self.alphabet)

    def encode(self, name: str, shift: bool = False) -> torch.Tensor:
        # улучшенный метод кодирования
        # усложненный сдвиг позволяет сохранить первый и последний символ исходного слова
        name = [*name, self.EOS]
        if not shift:
            name = [self.SOS, *name]
        indices = [self.ch2i[ch] for ch in name]
        indices += [self.PAD_IDX] * (self.max_len - len(indices))
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices: torch.Tensor) -> str:
        pad_indices = torch.nonzero(indices == self.ch2i[self.PAD], as_tuple=True)[0]
        if len(pad_indices):
            indices = indices[:pad_indices[0]]
        return "".join(self.alphabet[i] for i in indices)


class NamesDataset:
    names: t.List[str]
    vocab: NamesVocab
    data: torch.Tensor
    targets: torch.Tensor

    def __init__(self, path: Path):
        self.names = self.read_names(path)
        self.vocab = NamesVocab(self.names)

        self.data = torch.vstack([self.encode(name, shift=False) for name in self.names])
        self.targets = torch.vstack([self.encode(name, shift=True) for name in self.names])

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

    @staticmethod
    def read_names(path: Path) -> t.List[str]:
        with open(path, encoding="cp1251") as f:
            return list(map(lambda s: s.strip().lower(), f))

    def encode(self, name: str, shift: bool = False) -> torch.Tensor:
        return self.vocab.encode(name, shift=shift)

    def decode(self, vector: torch.Tensor) -> str:
        return self.vocab.decode(vector)

In [6]:
names_dataset = NamesDataset(DATA_DIR / "name_rus.txt")
print(f"n: {len(names_dataset)}")
(names_dataset.names[0], *names_dataset[0])

n: 1988


('авдокея',
 tensor([ 2, 26, 13, 20, 16,  7, 29, 22,  3,  0,  0,  0,  0,  0,  0]),
 tensor([26, 13, 20, 16,  7, 29, 22,  3,  0,  0,  0,  0,  0,  0,  0]))

Такой метод кодирования позволяет сохранить на одну букву больше, чем предложенный в задании - теряем `<SOS>`, но сохраняем первый и последний символ

In [7]:
torch.manual_seed(0)

train_names_dataset, test_names_dataset = train_test_split(names_dataset, train_part=0.8)
print(len(train_names_dataset), len(test_names_dataset))

1590 398


1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [8]:
class NamesRNNGenerator(nn.Module):
    _STATE_T = t.Union[t.Optional[torch.Tensor], t.Optional[t.Tuple[torch.Tensor, torch.Tensor]]]
    rnn_state: _STATE_T

    def __init__(
            self,
            num_embeddings: int,
            embedding_dim: int,
            rnn_hidden_size: int,
            rnn_cls: t.Union[t.Type[nn.RNN], t.Type[nn.LSTM], t.Type[nn.GRU]],
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.rnn = rnn_cls(input_size=embedding_dim, hidden_size=rnn_hidden_size)
        self.fc = nn.Sequential(
            nn.Linear(rnn_hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, num_embeddings),
        )
        self.reset_rnn_state()

    def reset_rnn_state(self):
        self.rnn_state = None

    def keep_rnn_state(self, state: _STATE_T):
        if isinstance(self.rnn, nn.LSTM):  # отдельная обработка скрытого состояния nn.LSTM
            self.rnn_state = state[0].detach(), state[1].detach()
        else:
            self.rnn_state = state.detach()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)

        x, rnn_state = self.rnn(x, self.rnn_state)
        self.keep_rnn_state(rnn_state)

        x = self.fc(x)
        # размерности отличаются от размерностей в задании:
        # [batch_size x кол-во символов x вероятности для каждого символа]
        # CrossEntropyLoss умеет так
        return x.permute(0, 2, 1)

In [9]:
# честная вероятность
def true_prob(pred: torch.Tensor) -> torch.Tensor:
    pred -= pred.min()
    return pred / pred.sum()


# вероятность через softmax - это не вероятность, сильный скос в сторону большего значения
def softmax_prob(pred: torch.Tensor) -> torch.Tensor:
    return torch.softmax(pred, 0)


def generate_name(
        model: NamesRNNGenerator,
        dataset: NamesDataset,
        prompt="",
        prob: t.Callable[[torch.Tensor], torch.Tensor] = None,
        device: str = "cpu",
) -> str:
    len_start = len(prompt)
    name = dataset.encode(prompt).to(device)
    name[len_start + 1] = 0  # заменяем <EOS> на <PAD>

    model.eval()
    model.reset_rnn_state()
    for i in range(name.size(0) - len_start - 2):
        # здесь передаем все слово, а не по 1-ой букве
        # не зря же модель учили словами, а не буквами...
        pred = model(name.unsqueeze(0)).squeeze()[:, len_start + i]
        if prob:  # случайность
            next_ch_idx = torch.multinomial(prob(pred), 1)
        else:  # честное взятие лучшего варианта
            next_ch_idx = pred.argmax()

        if next_ch_idx == NamesVocab.EOS_IDX:
            break
        name[len_start + i + 1] = next_ch_idx

    return dataset.decode(name).replace(NamesVocab.SOS, "")


def on_epoch_end_generate_names(
        model: NamesRNNGenerator,
        dataset: NamesDataset,
) -> t.Callable[[], None]:
    def _on_epoch_end() -> None:
        # честное взятие лучшего варианта
        const = generate_name(model, dataset, device=DEVICE)
        # случайное взятие на основе вероятности
        true_random = generate_name(model, dataset, prob=true_prob, device=DEVICE)
        # случайное взятие на softmax преобразования
        softmax_random = generate_name(model, dataset, prob=softmax_prob, device=DEVICE)
        print(f"\tNames: {const} (max), {true_random} (prob), {softmax_random} (softmax)")

    return _on_epoch_end

In [10]:
torch.manual_seed(0)

names_gen_net = NamesRNNGenerator(
    num_embeddings=len(names_dataset.vocab),
    embedding_dim=8,  # для символов большие embedding'и не нужны (наверное)
    rnn_hidden_size=64,
    rnn_cls=nn.LSTM,
).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(names_gen_net.parameters(), lr=0.001)

train_dataloader = DataLoader(train_names_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_names_dataset, batch_size=128)

In [11]:
%%time

_ = common_train(
    epochs=30,
    model=names_gen_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=50,
    on_epoch_end=on_epoch_end_generate_names(names_gen_net, names_dataset),
    device=DEVICE,
)

Epoch 1
--------------------------------
loss: 3.494831  [    0/ 1590]
Test Error: 
	Accuracy: 0.529816, Loss: 1.912179
	Names:  (max), аоа (prob), лпэьг (softmax)

Epoch 2
--------------------------------
loss: 1.987921  [    0/ 1590]
Test Error: 
	Accuracy: 0.578559, Loss: 1.577202
	Names: а (max), тале (prob), ы (softmax)

Epoch 3
--------------------------------
loss: 1.672587  [    0/ 1590]
Test Error: 
	Accuracy: 0.608710, Loss: 1.442052
	Names: а (max), ухйч (prob), с (softmax)

Epoch 4
--------------------------------
loss: 1.637822  [    0/ 1590]
Test Error: 
	Accuracy: 0.639531, Loss: 1.313473
	Names: а (max), руеьхилчзичй (prob), дабюшяй (softmax)

Epoch 5
--------------------------------
loss: 1.303850  [    0/ 1590]
Test Error: 
	Accuracy: 0.664824, Loss: 1.211651
	Names: а (max), нююо (prob), гтсыя (softmax)

Epoch 6
--------------------------------
loss: 1.203625  [    0/ 1590]
Test Error: 
	Accuracy: 0.677052, Loss: 1.144651
	Names: а (max),  (prob), нжаня (softmax)

Ep

In [12]:
y_test, y_pred = get_y_test_y_pred(names_gen_net, test_dataloader, DEVICE)

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=[names_dataset.vocab.alphabet[i] for i in y_test.unique().sort()[0]],
    zero_division=True,
))

              precision    recall  f1-score   support

       <PAD>       1.00      1.00      1.00      3110
       <EOS>       0.77      0.94      0.84       398
           у       1.00      0.02      0.03        62
           ф       1.00      0.00      0.00        12
           б       1.00      0.00      0.00        15
           к       0.21      0.11      0.15        98
           л       0.50      0.11      0.18       160
           с       0.28      0.07      0.11       104
           ш       0.27      0.34      0.30        56
           п       1.00      0.00      0.00        36
           и       0.21      0.23      0.22       183
           в       0.14      0.03      0.04        77
           т       0.36      0.07      0.11       118
           ы       1.00      0.00      0.00        20
           о       0.35      0.09      0.14        77
           э       1.00      0.00      0.00         6
           м       1.00      0.00      0.00        82
           ч       0.90    

Можно порадоваться за модель - она научилась успешно предсказывать `<PAD>` и `<EOS>`. Это была несложная задача

In [119]:
print(generate_name(names_gen_net, names_dataset, prompt="юн", device=DEVICE))
print(generate_name(names_gen_net, names_dataset, prompt="юн", prob=true_prob, device=DEVICE))
print(generate_name(names_gen_net, names_dataset, prompt="юн", prob=softmax_prob, device=DEVICE))

юната
юнаша
юна


Что это если не убийца ChatGPT?

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены `<SOS>`, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

В качестве датасета используется текст "Анна Каренина"

In [120]:
class TextVocab:
    PAD = "<PAD>"
    PAD_IDX = 0
    UNK = "<UNK>"
    UNK_IDX = 1

    def __init__(self, seqs: t.List[str]):
        uniques = set()
        max_len = 0
        for seq in map(str.lower, seqs):
            uniques.update(seq)
            max_len = max(len(seq), max_len)

        self.alphabet = [self.PAD, self.UNK, *uniques]
        self.max_len = max_len

        ch2i = {ch: i for i, ch in enumerate(self.alphabet)}
        self.ch2i = defaultdict(lambda: self.UNK_IDX, ch2i)

    def __len__(self):
        return len(self.alphabet)

    def encode(self, seq: str) -> torch.Tensor:
        indices = [self.ch2i[ch] for ch in seq]
        indices += [self.PAD_IDX] * (self.max_len - len(indices))
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices: torch.Tensor) -> str:
        pad_indices = torch.nonzero(indices == self.ch2i[self.PAD], as_tuple=True)[0]
        if len(pad_indices):
            indices = indices[:pad_indices[0]]
        return "".join(self.alphabet[i] for i in indices)


class TextDataset:
    seqs: t.List[str]
    vocab: TextVocab
    data: torch.Tensor
    targets: torch.Tensor

    def __init__(self, path: Path, window: int, overlap: int = 0):
        self.seqs = self.read_seqs(path, window=window, overlap=overlap)
        self.vocab = TextVocab(self.seqs)

        self.data = torch.vstack([self.encode(seq[:-1]) for seq in self.seqs])
        self.targets = torch.vstack([self.encode(seq[1:]) for seq in self.seqs])

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

    @staticmethod
    def read_seqs(path: Path, window: int, overlap: int = 0) -> t.List[str]:
        with open(path, encoding="cp1251") as f:
            text = list(map(lambda s: s.strip().lower(), f))

        text = " ".join(text)
        text = re.sub(r"[^а-яё]", repl=" ", string=text)
        text = " ".join(text.split())  # избавляемся от длинных пробельных последовательностей

        seqs = []
        for i in range(0, len(text), window):
            # последовательности длины window с перекрытием overlap с обоих сторон
            seqs.append(text[i:i + window + overlap])

        return seqs

    def encode(self, seq: str) -> torch.Tensor:
        return self.vocab.encode(seq)

    def decode(self, indices: torch.Tensor) -> str:
        return self.vocab.decode(indices)

In [121]:
text_dataset = TextDataset(DATA_DIR / "anna_karenina.txt", window=64, overlap=4)
print(f"n: {len(text_dataset)}")
(text_dataset.seqs[0], *text_dataset[0])

n: 25099


('лев николаевич толстой анна каренина мне отмщение и аз воздам часть ',
 tensor([ 7, 30, 13,  6, 31, 12,  8, 16,  7, 26, 30, 13, 12, 19,  6, 14, 16,  7,
          9, 14, 16, 24,  6, 26, 31, 31, 26,  6,  8, 26, 29, 30, 31, 12, 31, 26,
          6, 18, 31, 30,  6, 16, 14, 18,  2, 30, 31, 12, 30,  6, 12,  6, 26, 20,
          6, 13, 16, 20, 21, 26, 18,  6, 19, 26,  9, 14, 34,  0]),
 tensor([30, 13,  6, 31, 12,  8, 16,  7, 26, 30, 13, 12, 19,  6, 14, 16,  7,  9,
         14, 16, 24,  6, 26, 31, 31, 26,  6,  8, 26, 29, 30, 31, 12, 31, 26,  6,
         18, 31, 30,  6, 16, 14, 18,  2, 30, 31, 12, 30,  6, 12,  6, 26, 20,  6,
         13, 16, 20, 21, 26, 18,  6, 19, 26,  9, 14, 34,  6,  0]))

In [122]:
text_dataset.seqs[:10]

['лев николаевич толстой анна каренина мне отмщение и аз воздам часть ',
 'сть первая все счастливые семьи похожи друг на друга каждая несчастл',
 'астливая семья несчастлива по своему все смешалось в доме облонских ',
 'ких жена узнала что муж был в связи с бывшею в их доме француженкою ',
 'кою гувернанткой и объявила мужу что не может жить с ним в одном дом',
 ' доме положение это продолжалось уже третий день и мучительно чувств',
 'вствовалось и самими супругами и всеми членами семьи и домочадцами в',
 'ми все члены семьи и домочадцы чувствовали что нет смысла в их сожит',
 'ожительстве и что на каждом постоялом дворе случайно сошедшиеся люди',
 'люди более связаны между собой чем они члены семьи и домочадцы облон']

In [123]:
torch.manual_seed(0)

train_text_dataset, test_text_dataset = train_test_split(text_dataset, train_part=0.8)
print(len(train_text_dataset), len(test_text_dataset))

20079 5020


2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)


In [124]:
class TextRNNGenerator(nn.Module):
    _STATE_T = t.Union[t.Optional[torch.Tensor], t.Optional[t.Tuple[torch.Tensor, torch.Tensor]]]
    rnn_state: _STATE_T

    def __init__(
            self,
            num_embeddings: int,
            embedding_dim: int,
            rnn_hidden_size: int,
            rnn_cls: t.Union[t.Type[nn.RNN], t.Type[nn.LSTM], t.Type[nn.GRU]],
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        # применить двухуровневые рекуррентные слои можно,
        # было бы хорошо, если это как-то влияло на точность ну хоть чуть-чуть
        self.rnn = rnn_cls(input_size=embedding_dim, hidden_size=rnn_hidden_size, num_layers=2, dropout=0.25)
        self.fc = nn.Sequential(
            nn.Linear(rnn_hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, num_embeddings),
        )
        self.reset_rnn_state()

    def reset_rnn_state(self):
        self.rnn_state = None

    def keep_rnn_state(self, state: _STATE_T):
        if isinstance(self.rnn, nn.LSTM):
            self.rnn_state = state[0].detach(), state[1].detach()
        else:
            self.rnn_state = state.detach()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)

        x, rnn_state = self.rnn(x, self.rnn_state)
        self.keep_rnn_state(rnn_state)

        x = self.fc(x)
        return x.permute(0, 2, 1)

In [125]:
torch.manual_seed(0)

text_gen_net = TextRNNGenerator(
    num_embeddings=len(text_dataset.vocab),
    embedding_dim=16,
    rnn_hidden_size=64,
    rnn_cls=nn.LSTM,
).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(text_gen_net.parameters(), lr=0.001)

train_dataloader = DataLoader(train_text_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_text_dataset, batch_size=512)

In [126]:
%%time

_ = common_train(
    epochs=10,
    model=text_gen_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=200,
    device=DEVICE,
)

Epoch 1
--------------------------------
loss: 3.563401  [    0/20079]
loss: 2.622643  [12800/20079]
Test Error: 
	Accuracy: 0.256237, Loss: 2.550842

Epoch 2
--------------------------------
loss: 2.585614  [    0/20079]
loss: 2.513025  [12800/20079]
Test Error: 
	Accuracy: 0.267832, Loss: 2.458726

Epoch 3
--------------------------------
loss: 2.482423  [    0/20079]
loss: 2.491698  [12800/20079]
Test Error: 
	Accuracy: 0.269589, Loss: 2.429040

Epoch 4
--------------------------------
loss: 2.458759  [    0/20079]
loss: 2.475520  [12800/20079]
Test Error: 
	Accuracy: 0.270081, Loss: 2.422082

Epoch 5
--------------------------------
loss: 2.470808  [    0/20079]
loss: 2.416837  [12800/20079]
Test Error: 
	Accuracy: 0.270081, Loss: 2.419302

Epoch 6
--------------------------------
loss: 2.436810  [    0/20079]
loss: 2.431002  [12800/20079]
Test Error: 
	Accuracy: 0.269844, Loss: 2.417486

Epoch 7
--------------------------------
loss: 2.424822  [    0/20079]
loss: 2.452827  [12800/

Ну тут либо постановка задачи невыполнимая, либо модель не обучается

In [127]:
y_test, y_pred = get_y_test_y_pred(text_gen_net, test_dataloader, DEVICE)

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=[text_dataset.vocab.alphabet[i] for i in y_test.unique().sort()[0]],
    zero_division=True,
))

              precision    recall  f1-score   support

       <PAD>       1.00      1.00      1.00      5020
           щ       1.00      0.00      0.00       807
           у       1.00      0.00      0.00      7684
           ф       1.00      0.00      0.00       289
           б       1.00      0.00      0.00      4962
                   0.32      0.79      0.46     56390
           л       1.00      0.00      0.00     14217
           к       1.00      0.00      0.00      9621
           с       0.10      0.37      0.15     14917
           п       1.00      0.00      0.00      6787
           ш       1.00      0.00      0.00      2446
           и       1.00      0.00      0.00     18485
           в       1.00      0.00      0.00     13055
           т       0.29      0.35      0.32     16653
           ы       0.29      0.28      0.28      5149
           о       0.30      0.47      0.37     32323
           э       1.00      0.00      0.00      1008
           м       1.00    

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3

In [128]:
def generate_text(
        model: TextRNNGenerator,
        dataset: TextDataset,
        prompt: str,  # стартовая строка
        size: int,  # любая длина генерируемого текста
        prob: t.Callable[[torch.Tensor], torch.Tensor] = None,
        device: str = "cpu",
) -> str:
    text = [dataset.vocab.ch2i[ch] for ch in prompt]

    model.eval()
    model.reset_rnn_state()
    for i in range(size - len(text)):
        x = torch.tensor([text[i:]], device=device)
        pred = model(x).squeeze()[:, -1]
        if prob:
            next_ch_idx = torch.multinomial(prob(pred), 1)
        else:
            next_ch_idx = pred.argmax()
        text.append(next_ch_idx.item())

    return dataset.decode(torch.tensor(text))

In [129]:
for prompt in [
    "доброе утро ",
    "добрый вечер ",
    "на каждом постоялом дворе ",
    "дело шло о том ",
    "как только ",
    "белая береза ",
]:
    print(generate_text(text_gen_net, text_dataset, prompt, 90, prob=softmax_prob, device=DEVICE))

доброе утро жда ем еналаи кицу блсетаю после стьскромол ела одвснееенидевыесевоже од икс т
добрый вечер гомаковшиск ка з предего инубествери мыт тупепретовос те см нытлоскокнай азас
на каждом постоялом дворе спровиполю вонобло подрину фре деля в атото еме ка дь стви ду вы
дело шло о том б нс канн по ей е тару чкрия пув монтри чевровой не ито пренака эт вожата м
как только ро ола заза выкста поерыепри бкодру сти ий нытодошьзн пре о вытулю лен усделялю
белая береза гол жче яшь онелодрго стамо мо пу иниль ко столю чирыло овылиедл к ситотенолу


А это "Убийца ChatGPT 2.0"

> As we can see, the generated text may not make any sense, however there are some words and phrases that seem to form an idea, for example...

[Источник](https://towardsdatascience.com/text-generation-with-bi-lstm-in-pytorch-5fda6e7cc22c#:~:text=As%20we%20can%20see%2C%20the%20generated%20text%20may%20not%20make%20any%20sense%2C%20however%20there%20are%20some%20words%20and%20phrases%20that%20seem%20to%20form%20an%20idea%2C%20for%20example%3A)

Из хорошего, модель научилась вовремя расставлять пробелы - сочетания букв по длинам похожи на слова (самостоятельные и предлоги - если сильно фантазировать 🥳)